In [16]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import os

In [17]:
# open csv from part 1
city_data_df = pd.read_csv(os.path.join("weather_data","WeatherPy_challenge.csv"))
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date
0,0,Qaanaaq,77.48,-69.36,-14.51,58,0,5.61,GL,clear sky,0.00,0.0,2020-02-20 22:46:30
1,1,Hobart,-42.88,147.33,61.00,44,40,13.87,AU,light rain,0.25,0.0,2020-02-20 22:46:30
2,2,Albany,42.60,-73.97,26.60,38,1,16.11,US,clear sky,0.00,0.0,2020-02-20 22:45:03
3,3,Port Alfred,-33.59,26.89,68.00,92,59,6.04,ZA,broken clouds,0.00,0.0,2020-02-20 22:46:31
4,4,Najran,17.49,44.13,73.40,35,53,4.70,SA,broken clouds,0.00,0.0,2020-02-20 22:46:32


In [19]:
#create separate df's from the csv
paloalto_city_df = city_data_df.loc[city_data_df["City"] == "Palo Alto"]
hillsborough_city_df = city_data_df.loc[city_data_df["City"] == "Hillsborough"]
halfmoon_city_df = city_data_df.loc[city_data_df["City"] == "Half Moon Bay"]
pacificgrove_city_df = city_data_df.loc[city_data_df["City"] == "Pacific Grove"]
selma_city_df = city_data_df.loc[city_data_df["City"] == "Selma"]

In [26]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

## Map figure with directions layer

In [70]:
# Latitude-longitude pairs
paloalto_lat_long = tuple([x for x in paloalto_city_df[["Lat","Lng"]].to_records(index=False)][0])
hillsborough_lat_long = tuple([x for x in hillsborough_city_df[["Lat","Lng"]].to_records(index=False)][0])
halfmoon_lat_long = tuple([x for x in halfmoon_city_df[["Lat","Lng"]].to_records(index=False)][0])
pacificgrove_lat_long = tuple([x for x in pacificgrove_city_df[["Lat","Lng"]].to_records(index=False)][0])
selma_lat_long = tuple([x for x in selma_city_df[["Lat","Lng"]].to_records(index=False)][0])

# Create the figure with directions layer
fig = gmaps.figure()
norcal_directions_layer = gmaps.directions_layer(paloalto_lat_long,
                                                 selma_lat_long,
                                                 waypoints=[hillsborough_lat_long,halfmoon_lat_long,pacificgrove_lat_long],
                                                 travel_mode="DRIVING")
fig.add_layer(norcal_directions_layer)
fig

Figure(layout=FigureLayout(height='420px'))